In [1]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic
from rich.pretty import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import h5py
import os

In [11]:
def download_month(month: int, year: int, start_day = 1, cached=True):

    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)

    filename = "Frankfurt_LH_" + str(year)[2:] + str_month + ".h5"
    print(filename)
    next_month = month + 1 if month < 12 else  1
    str_next_month = str(next_month) if next_month > 9 else "0" + str(next_month)

    start_day_str = "0" + str(start_day) if start_day < 10 else str(start_day)
    datetime_str_start = str(year) + "-"+ str_month + "-" + start_day_str +  " 00:00:00"
    datetime_str_end = str(year) + "-" + str_next_month + "-01 00:00:00"
    datetime_current = datetime.strptime(datetime_str_start, '%Y-%m-%d %H:%M:%S')
    datetime_end = datetime.strptime(datetime_str_end, '%Y-%m-%d %H:%M:%S')

    while datetime_current < datetime_end:

        print(datetime_current)
        day = datetime_current.day
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)

        h5_key = "LH_22" + str_month + day_str
        print(h5_key)
        str_current_day = str(datetime_current)
        str_next_day = str(datetime_current + timedelta(days = 1))
        print("getting data between "+str_current_day+ " and "+ str_next_day)

        trajectories = opensky.history(str_current_day,
                            stop= str_next_day,
                            arrival_airport ="EDDF",
                            cached=cached)
        try:
            callsigns_lh = trajectories.data.callsign.loc[trajectories.data.callsign.str.contains("DLH").astype(bool)]

            callsigns_lh = callsigns_lh.unique()
            callsigns_lh = callsigns_lh[~pd.isnull(callsigns_lh)]
            trajectories_lh = trajectories[callsigns_lh]
            print("add",h5_key,"to file")

            trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

        except AttributeError as e:
            print(e)

        datetime_current += timedelta(days = 1)


In [ ]:
download_month(9, 2022, 26, cached = True)

In [22]:
trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

In [12]:
with h5py.File("Frankfurt_LH_2209.h5", 'r') as f:
    print(list(f.keys()))

['LH_220901', 'LH_220902', 'LH_220903', 'LH_220904', 'LH_220905', 'LH_220906', 'LH_220907', 'LH_220908', 'LH_220909', 'LH_220910', 'LH_220911', 'LH_220912', 'LH_220913', 'LH_220914', 'LH_220915', 'LH_220916', 'LH_220917', 'LH_220918', 'LH_220919', 'LH_220920', 'LH_220921', 'LH_220922', 'LH_220923', 'LH_220924', 'LH_220925', 'LH_220926', 'LH_220927', 'LH_220928', 'LH_220929']
